In [ ]:
import numpy as np 
import pandas as pd 
import mf
from nfc.src.mlp import MLP as mlp

f = open('MSD/train_triplets.txt', 'r')
song_to_count = dict()
songs_per_user = dict()
user_to_songs = dict() 
for line in f:
    user,song,_ = line.strip().split('\t')
    if song in song_to_count:
        song_to_count[song]+=1
    else:
        song_to_count[song] = 1
    if user in user_to_songs:
        user_to_songs[user].add(song)
    else:
        user_to_songs[user] = set([song])
    if user in songs_per_user:
        songs_per_user[user]+=1
    else:
        songs_per_user[user] = 1
f.close()

# Data preprocessing - Only keep users with more than 20 songs and songs that have been listened by at least 200 users!
valid_songs = set()
user_to_songs = {k: v for k, v in user_to_songs.items() if songs_per_user[k]>20}
for user,user_songs in user_to_songs.items():
    new_list = [s for s in list(user_songs) if song_to_count[s] >= 200]
    user_to_songs[user] = new_list
    for valid_song in new_list:
        valid_songs.add(valid_song)
user_to_songs = {k: v for k, v in user_to_songs.items() if v}

# Creating sparse matrix Coordinate Format (COO)

users = {}
songs = {}
i = 0 # Index for users
for user in user_to_songs.keys():
    users[i] = user
    i+=1
j = 0 # index for songs
for song in valid_songs:
    songs[j] = song
    j+=1

In [2]:
from scipy.sparse import coo_matrix,save_npz

row = []
column = []
data = []
for user_index,user in users.items():
    for song_index,song in songs.items():
        row.append(user_index)
        column.append(song_index)
        data.append(1 if (song in user_to_songs[user]) else 0)

In [3]:
from mf import MF 
row = np.array(row)
column = np.array(column)
data = np.array(data)
B = coo_matrix((data, (row, column))).toarray()
mf = MF(B, 5, 0.05, 1, 20)

93372


In [1]:
import os
import glob
import hdf5_getters

def get_all_titles(basedir,ext='.h5') :
    titles = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            h5 = hdf5_getters.open_h5_file_read(f)
            titles.append( hdf5_getters.get_title(h5) )
            h5.close()
    return titles

def count_all_files(basedir,ext='.h5') :
    cnt = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        cnt += len(files)
    return cnt

In [5]:
titles = get_all_titles('MSD')

In [4]:
titles

[b'Deep Sea Creature',
 b"Kablammin' It",
 b"It's In The Book (Album Version)",
 b'Le rire du sergent',
 b'See You On The Other Side',
 b'Ballo',
 b'Sex (French Version)',
 b'United Blood (Album Version)',
 b'Le poulet',
 b"Don't Mess With Cupid (1972 demo)",
 b'Over The Rainbow',
 b'Killed By Love',
 b'No Safe Place',
 b'Daydreaming the Color of Blood',
 b'Boss-Eyed Whelk',
 b'All Mama Ever Wanted  (Album)',
 b'Battle Hymn Of The Republic',
 b'Teri Nau Ke Paune Teen',
 b'Real World',
 b'Canyon',
 b'Supertouch/shitfit (Bad Brains)',
 b'Liu Zi Zhen Yan',
 b'Who Do You Love',
 b'White Christmas',
 b'The Hardway',
 b'Skate alone',
 b"Dodo's Bounce",
 b'Babylon 5',
 b'Revolution Action',
 b'Up Jumped The Devil',
 b'Free Africa',
 b'Killer Barracuda',
 b'The Date ( LP Version )',
 b'Noche Eterna',
 b'Seventeen',
 b'Poes\xc3\xada De Amor',
 b"Take Me Fishin'",
 b'Give It To Me',
 b'The Coming Of The King',
 b'She (1997 Digital Remaster)',
 b'Frente A Tu Altar',
 b"If You Don't Somebody Else 